----
# <center>Dataset creation

In [62]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import shutil
import tempfile
import json

base_dir = Path("./")

 # --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

 # --- Add SCOPE path to MATLAB's search path ---
scope_path = './SCOPE'  # e.g., '/home/user/Documents/SCOPE'
eng.addpath(scope_path, nargout=0)

# MODTRAN paths
modtran_dir = base_dir / "MODTRAN5"
modtran_exe = modtran_dir / "bin" / "Mod5_mac.exe" 
modtran_tp5_template = modtran_dir / "HyPlant-FLUO_Modtran5_base_v1.tp5"

# SCOPE paths
scope_dir = base_dir / "SCOPE"
scope_main = scope_dir / "SCOPE.m"
scope_wrapper = scope_dir / "run_scope_wrapper.m"

# Output directory
output_dir = base_dir / "synthetic_dataset"
output_dir.mkdir(exist_ok=True)

print(f"Checking path: {scope_dir.resolve()}")
print(f"Directory exists: {scope_dir.exists()}")
print(f"Directory contents: {list(scope_dir.glob('*')) if scope_dir.exists() else 'N/A'}")


Checking path: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/SCOPE
Directory exists: True
Directory contents: [PosixPath('SCOPE/.DS_Store'), PosixPath('SCOPE/run_scope_wrapper_json.m'), PosixPath('SCOPE/input'), PosixPath('SCOPE/output'), PosixPath('SCOPE/docs'), PosixPath('SCOPE/soltir_tp7.m'), PosixPath('SCOPE/SCOPE.m'), PosixPath('SCOPE/README.md'), PosixPath('SCOPE/set_parameter_filenames.csv'), PosixPath('SCOPE/bug_reports.txt'), PosixPath('SCOPE/GNU_General_Public_Licence.txt'), PosixPath('SCOPE/.git'), PosixPath('SCOPE/.readthedocs.yaml'), PosixPath('SCOPE/SCOPE.exe'), PosixPath('SCOPE/src')]


In [5]:
from scopeWrapper import SCOPEWrapperMultiRun

if __name__ == "__main__":
    print("\n=== Path Diagnostics ===")
    print(f"Current working directory: {Path.cwd()}")
    print(f"Checking for 'SCOPE' in: {Path.cwd()}")

    # the spectral and scalar files that will be saved in the results.json
    
    spectral_files = {
        #"Eout_spectrum": "Eout_spectrum.csv",
        #"Lo_spectrum": "Lo_spectrum.csv", 
        "fluorescence": "fluorescence.csv",
        #"sigmaF": "sigmaF.csv",
        "reflectance": "reflectance.csv",
    }
    
    scalar_files = {
        #"aPAR": "aPAR.csv",
        #"Eout": "Eout.csv",
        #"Lo": "Lo.csv",
        #"Esun": "Esun.csv",
        #"Esky": "Esky.csv",
        #"fluxes": "fluxes.csv",
        #"rad": "radiation.csv",
        #"fluorescence_scalars": "fluorescence_scalars.csv",
    }

    try:
        with SCOPEWrapperMultiRun(scalar_files=scalar_files, spectral_files=spectral_files) as scope:
            print("\n=== Running Multi-Simulation Test ===")
            # Example parameters (can be adjusted)

            # Run a simulation with multiple LAI and Cab values
            multi_params = {
                # PROSPECT leaf optical properties
                "Cab": [30.0, 40.0, 50.0],      # Chlorophyll content (μg/cm²)
                "Cca": [10.0],                  # Carotenoid content (μg/cm²)
                "Cdm": [0.012],                 # Dry matter content (g/cm²)
                "Cw": [0.009],                  # Equivalent water thickness (cm)
                "Cs": [0.0],                    # Senescent material fraction (0-1)
                "Cant": [1.0],                  # Anthocyanin content (μg/cm²)
                "N": [1.5],                     # Leaf structure parameter (-)

                # Leaf biochemical parameters
                "Vcmax25": [60.0],              # Maximum carboxylation rate at 25°C (μmol/m²/s)
                "BallBerrySlope": [8.0],        # Ball-Berry stomatal conductance slope
                "BallBerry0": [0.01],           # Ball-Berry intercept (mol/m²/s)

                # Canopy structure
                "LAI": [2.0, 3.0, 5.0],        # Leaf Area Index (m²/m²)
                "hc": [2.0],                    # Canopy height (m)
                "LIDFa": [-0.35],               # Leaf angle distribution parameter a (-)
                "LIDFb": [-0.15],               # Leaf angle distribution parameter b (-)

                # Soil parameters
                "rss": [500.0],                 # Soil respiration rate (μmol/m²/s)
                "SMC": [25.0],                  # Soil moisture content (%)
                
                # Meteorology
                "Rin": [800.0],                 # Incoming shortwave radiation (W/m²)
                "Ta": [20.0],                   # Air temperature (°C)
                "Ca": [410.0],                  # Atmospheric CO₂ concentration (ppm)

                # Fluorescence
                "fqe": [0.01, 0.03, 0.06, 0.1, 0.2], # Fluorescence quantum efficiency (-)

                # Angles
                "tts": [35.0, 0.00, 0.60],                  # Solar zenith angle (degrees)
                "tto": [0.0],                   # Observer zenith angle (degrees)
                "psi": [0.0]                    # Relative azimuth angle (degrees)
            }

            my_setoptions = {
                "simulation": 2,  # Enable Lookup-Table mode
                "calc_fluor": 1,  # Calculate fluorescence
                "soil_heat_method": 2,  # Use soil net radiation fraction
                "saveCSV": 1,  # Output CSV files
                "calc_planck": 0,
                "calc_directional": 0,
                "calc_planck": 0,
                "calc_vert_profiles": 0,
                "calc_rss_rbs": 0,
                "soil_heat_method": 2,
                "save_spectral": 1,
                "calc_xanthophyllabs": 0,
                "applTcorr": 0,
                "MoninObukhov" : 0,
                "lite": 0,
                
            }

            results_path = scope.run(multi_params, my_setoptions)

            print("Success!")
            with open(results_path, "r") as f:
                results = json.load(f)

            print("\n=== Results ===")

            print(
                f"Number of simulations in output: {results.get('num_simulations')}"
            ) 

    except Exception as e:
        print(f"Error during SCOPE execution: {e}")


=== Path Diagnostics ===
Current working directory: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project
Checking for 'SCOPE' in: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project

=== Running Multi-Simulation Test ===
=== Parameters before CSV generation ===
{
  "Cab": [
    30.0,
    40.0,
    50.0
  ],
  "Cca": [
    10.0
  ],
  "Cdm": [
    0.012
  ],
  "Cw": [
    0.009
  ],
  "Cs": [
    0.0
  ],
  "Cant": [
    1.0
  ],
  "N": [
    1.5
  ],
  "Vcmax25": [
    60.0
  ],
  "BallBerrySlope": [
    8.0
  ],
  "BallBerry0": [
    0.01
  ],
  "LAI": [
    2.0,
    3.0,
    5.0
  ],
  "hc": [
    2.0
  ],
  "LIDFa": [
    -0.35
  ],
  "LIDFb": [
    -0.15
  ],
  "rss": [
    500.0
  ],
  "SMC": [
    25.0
  ],
  "Rin": [
    800.0
  ],
  "Ta": [
    20.0
  ],
  "Ca": [
    410.0
  ],
  "fqe": [
    0.01,
    0.03,
    0.06,
    0.1,
    0.2
  ],
  "tts": [
    35.0,
    0.0,
    0.6
  ],
  "tto": [
    0.0
  ],
  "psi": [
    0.0
  ]
}

In [39]:
print(results.keys())
print(results["spectral_outputs"].keys())
print(results["input_parameters"].keys())
# print recursive keys
def print_keys(d, prefix=""):
    for k, v in d.items():
        print(f"{prefix}{k}")
        if isinstance(v, dict):
            print_keys(v, prefix=prefix + "-")
print_keys(results)

dict_keys(['scalar_outputs', 'run_parameters', 'num_simulations', 'input_parameters', 'setoptions', 'model_parameters', 'spectral_outputs', 'wlF', 'wlS'])
dict_keys(['fluorescence', 'reflectance'])
dict_keys(['PROSPECT', 'Leaf_Biochemical', 'Leaf_Biochemical_magnani', 'Fluorescence', 'Soil', 'Canopy', 'Meteo', 'Aerodynamic', 'timeseries', 'Angles'])
scalar_outputs
run_parameters
num_simulations
input_parameters
-PROSPECT
--Cab
--Cca
--Cdm
--Cw
--Cs
--Cant
--Cp
--Cbc
--N
--rho_thermal
--tau_thermal
-Leaf_Biochemical
--Vcmax25
--BallBerrySlope
--BallBerry0
--Type
--kV
--Rdparam
--Kn0
--Knalpha
--Knbeta
-Leaf_Biochemical_magnani
--Tyear
--beta
--kNPQs
--qLs
--stressfactor
-Fluorescence
--fqe
-Soil
--spectrum
--rss
--rs_thermal
--cs
--rhos
--lambdas
--SMC
--BSMBrightness
--BSMlat
--BSMlon
-Canopy
--LAI
--hc
--LIDFa
--LIDFb
--leafwidth
--Cv
--crowndiameter
-Meteo
--z
--Rin
--Ta
--Rli
--p
--ea
--u
--Ca
--Oa
-Aerodynamic
--zo
--d
--Cd
--rb
--CR
--CD1
--Psicor
--CSSOIL
--rbs
--rwc
-timeseries


In [65]:
import os
from pathlib import Path
import matlab.engine
import numpy as np
import json
from scipy.interpolate import interp1d
import pandas as pd

# --- Configuration ---
BASE_DIR = Path("./")
MODTRAN_MATLAB_DIR = BASE_DIR / "MODTRAN5"
SCOPE_OUTPUT_FILE = BASE_DIR / "synthetic_dataset" / "results.json"
OUTPUT_DIR = BASE_DIR / "output"
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# --- Start MATLAB Engine ---
eng = matlab.engine.start_matlab()

# --- Add MODTRAN paths to MATLAB (only needed once per session) ---
eng.addpath(str(MODTRAN_MATLAB_DIR), nargout=0)
eng.addpath(eng.genpath(str(MODTRAN_MATLAB_DIR / "modtran5_acd_t14")), nargout=0)
eng.addpath(eng.genpath(str(MODTRAN_MATLAB_DIR / "matlab-modtran-5-aba70d781805")), nargout=0)
print("MATLAB paths added successfully")

# --- Helper Functions ---

def run_modtran_matlab(parms, output_dir, case_name):
    """Runs MODTRAN via MATLAB Engine, handling errors gracefully."""
    try:
        # Convert nested Python dict to MATLAB struct properly
        matlab_parms = eng.struct()
        for key, value in parms.items():
            if isinstance(value, dict):
                sub_struct = eng.struct()
                for sub_key, sub_value in value.items():
                    sub_struct[sub_key] = sub_value
                matlab_parms[key] = sub_struct
            else:
                matlab_parms[key] = value

        # Call MODTRAN MATLAB function  (Use the CORRECTED version)
        wvlLUT, T14, _ = eng.run_modtran_corrected( #MODIFIED MATLAB FUNCTION
            str(output_dir),  # Pass as string
            case_name,
            matlab_parms,
            'FLUO',
            'A',
            1,  # H2O scaling flag
            0,  # CO2 scaling flag
            0,  # O3 scaling flag
            nargout=2  # MUST be 2 to get wvlLUT and T14
        )
        return np.array(wvlLUT), np.array(T14), None  # DV not used
    except matlab.engine.MatlabExecutionError as e:
        print(f"MATLAB/MODTRAN error: {e}")
        return None, None, None
    except FileNotFoundError:
        print(f"MODTRAN function 'run_modtran_corrected' not found.  Check MATLAB path.")
        return None, None, None  # Or raise the exception if you want to halt
    except Exception as e:
        print(f"Unexpected error: {e}")
        return None, None, None

def interpolate_and_combine(wl_scope, data_scope, wl_modtran):
    """Interpolates SCOPE data to MODTRAN wavelengths."""
    interpolator = interp1d(wl_scope, data_scope, kind='linear', bounds_error=False, fill_value="extrapolate")
    return interpolator(wl_modtran)


if __name__ == "__main__":
    # --- Load SCOPE Results ---
    try:
        with open(SCOPE_OUTPUT_FILE, 'r') as f:
            scope_results = json.load(f)
    except FileNotFoundError:
        print(f"Error: SCOPE output file '{SCOPE_OUTPUT_FILE}' not found.")
        exit(1)
    except json.JSONDecodeError:
        print(f"Error: Could not decode JSON in '{SCOPE_OUTPUT_FILE}'. Check for syntax errors.")
        exit(1)

    # --- Get Simulation Parameters ---
    num_simulations = scope_results['num_simulations']
    print(f"Number of simulations: {num_simulations}")

    # --- Access parameters SAFELY with error handling, from run_parameters ---
    def get_param(run_params, param_path, default_value=None):
        """Safely gets a parameter from a single run's parameters."""
        try:
            value = run_params
            for key in param_path:
                value = value[key]
            return value  # Return the actual value (could be scalar or list)
        except KeyError:
            if default_value is not None:
                print(f"Warning: Parameter '{'.'.join(param_path)}' not found. Using default: {default_value}")
                return default_value  # Return the default value
            else:
                print(f"Error: Parameter '{'.'.join(param_path)}' not found in SCOPE output.")
                exit(1)
        except TypeError as e:
            print(f"Error accessing parameter '{'.'.join(param_path)}': {e}. Check 'results.json' structure.")
            exit(1)

    # Extract relevant data from SCOPE results
    wlS = np.array(scope_results['wlS'])
    wlF = np.array(scope_results['wlF'])
    reflectance_data = np.array(scope_results['spectral_outputs']['reflectance'])
    fluorescence_data = np.array(scope_results['spectral_outputs']['fluorescence'])

     # Ensure 2D arrays
    if reflectance_data.ndim == 1:
        reflectance_data = reflectance_data.reshape(1, -1)
    if fluorescence_data.ndim == 1:
        fluorescence_data = fluorescence_data.reshape(1, -1)

    # --- Main Loop ---
    for i in range(num_simulations):

        # --- Get parameters for *this* simulation run ---
        run_params = scope_results["run_parameters"][i]

        # Get parameters, handling defaults.  Crucially, get them from *run_params*.
        gndalt = get_param(run_params, ["Canopy", "hc"], default_value=0.01)
        tts = get_param(run_params, ["Angles", "tts"], default_value=30.0)
        tto = get_param(run_params, ["Angles", "tto"], default_value=0.0)
        psi = get_param(run_params, ["Angles", "psi"], default_value=0.0)
        # You MUST get AOT and H2O from your SCOPE output if they are there.
        # If not, provide reasonable defaults or load them from elsewhere.
        aot = get_param(run_params, ["Meteo", "AOT"], default_value=0.1)  # Example default
        h2o = get_param(run_params, ["Meteo", "H2O"], default_value=0.5)   # Example default
        o3 = get_param(run_params, ["Meteo", "O3"], default_value = 'DEFAULT') #Added O3 parameter

        # --- 1. Prepare MODTRAN PARMS (REBUILD relevant parts inside the loop) ---
        PARMS = {
            'CASE': 0,
            'MODTRN': 'M',  # 'M' for MODTRAN, 'T' for high-resolution (slower)
            'SPEED': 'S',   # 'S' for slow, 'M' for medium
            'LYMOLC': 0,
            'MODEL': 2,     # Atmospheric model (2 = Mid-Latitude Winter)
            'ITYPE': 3,     # Slant path to space
            'IEMSCT': 3,    # Solar irradiance
            'IMULT': 0,     # Single scattering (DISORT: -1, 0, or +1)
            'M1': 0,        # No user-defined temperature/pressure
            'M2': 0,        # No user-defined H2O
            'M3': 0,        # No user-defined O3
            'M4': 0,        # No user-defined CH4
            'M5': 0,        # No user-defined N2O
            'M6': 0,        # No user-defined CO
            'MDEF': 0,      # Use default profiles for other species
            'I_RD2C': 0,
            'NOPRNT': 0,    # Print standard output
            'TPTEMP': 20.0,    # Blackbody temperature of the ground.
            'SURREF': 'LAMB_1', # Lambertian surface
            'DIS': '1',      # Use DISORT (if IMULT > 0)
            'DISAZM': '0',   # No DISORT azimuth dependence
            'DISALB': ' ',   # No special DISORT albedo calculation
            'NSTR': 8,      # Number of DISORT streams
            'SFWHM': 1.0,     # Spectral FWHM (cm-1)
            'CO2MX': 410.0,  # CO2 mixing ratio (ppmv)
            'LSUNFL': 1,    # Use default solar irradiance
            'LBMNAM': 0,    # Use default band model
            'WSS': 0.0,     # Wind speed (if using Navy Maritime aerosols)
            'WDS': 0.0,     # Wind direction (if using Navy Maritime aerosols)
            'IDAY': 191,   # Day of year (or get from SCOPE 'startDOY')
            'GMTIME': 12.0,   # Time of day (or get from SCOPE)
            'NSSALB': 0,
            'IPARM': 11,     # Geometry type (11 = solar angles)
            'APLUS' : ' ',
            'CNOVAM': ' ',
            'ARUSS': ' ',
            'SOLCON': ' ',
            'LFLTNM': ' ',
            'WSS': 0.0,  # Wind speed (m/s)
            'WDS': 0.0,  # Wind direction (degrees)
            
           'ATM': {  # Nested dictionary for ATM parameters
                'G': 0.7,           # Aerosol asymmetry factor (example)
                'AOT': aot,      # Aerosol optical thickness
                'CDASTM': 'DEFAULT',# Use default aerosol settings
                'ASTMX': 0.0,      # Angstrom exponent (if applicable)
                'CMULT': 1.0,      # Multiple scattering flag
                'MODEL': 'MIDLAT_WINTER',
                'IHAZE': 0,       # Aerosol model (0 = no aerosols)
                'H2OSTR': h2o, # Water vapor scaling (string!)
                'O3STR': 'DEFAULT',
                'C_PROF': ' ',
                'H2OAER' : ' ',
                'CDTDIR' : ' ',
                'ASTMO' : ' ',
                'AERRH': ' ',
                'IPH' : 2,
                'ISEASN': 0,          # Season code (0=default,1=winter,2=summer)
                'IVULCN': 0,          # Volcanic aerosols model
                'ICSTL': 1,           # Coastal aerosols (1=default)
                'ICLD': 0,            # Cloud model (0=no clouds)
                'IVSA': 0,            # Vertical structure aerosol option
            },
            'WAVELEN': { # Nested dictionary for WAVELEN parameters
                'WMIN': 650,
                'WMAX': 800,
                'RES': 1.0
            },
           'GEOM': {  # Nested dictionary for GEOM parameters
                'ITYPE': 3,
                'H1ALT': gndalt,  # Ground/target altitude (km)
                'OBSZEN': 180.0 - tto,   # Observer zenith angle (degrees)
                'TRUEAZ': psi + 180.0,  # Relative azimuth angle (degrees)
            },
            'PARM1': tts,  # Solar zenith angle (degrees) from SCOPE
            'PARM2': 0.0,  # Will be updated
            'VIS': 23.0,
            'RAINRT' : 0,
            'GNDALT' : gndalt,
            'T_BEST' : ' ' # Add T_BEST here!  Crucial! Use a space.
        }

        # --- 2. Run MODTRAN ---
        case_name = f"simulation_{i}"
        wvlLUT, T14, DV = run_modtran_matlab(PARMS, str(OUTPUT_DIR), case_name)

        if wvlLUT is None:
            print(f"Skipping simulation {i} due to MODTRAN error.")
            continue

        # --- 3. Interpolate SCOPE Data ---
        # Extract relevant data from SCOPE results
        wlS = np.array(scope_results['wlS'])
        wlF = np.array(scope_results['wlF'])
        reflectance_data = np.array(scope_results['spectral_outputs']['reflectance'])
        fluorescence_data = np.array(scope_results['spectral_outputs']['fluorescence'])

        # Ensure 2D arrays
        if reflectance_data.ndim == 1:
            reflectance_data = reflectance_data.reshape(1, -1)
        if fluorescence_data.ndim == 1:
            fluorescence_data = fluorescence_data.reshape(1, -1)

        R = interpolate_and_combine(wlS, reflectance_data[i, :], wvlLUT)
        F = interpolate_and_combine(wlF, fluorescence_data[i, :], wvlLUT)

        # --- 4. Calculate LBOA, LWLR, rho_app, and LTOA ---
        # Extract T14 values (make sure order is correct!)
        t_values = {f't{j}': T14[:, j-1] for j in range(1, 15)}
        t1, t2, t3, t4, t5, t6, t7, t8, t9, t10, t11, t12 = [t_values[f't{j}'] for j in [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]]

        # Calculate LBOA (Equation from the professor)
        LBOA = t1 * (t4 + t5) * R + t1*((t9 * R + t10* R + t11 * R)/(1 - R * t3)) + t1 * (t6 * F + t7 * F)

        # Calculate LWLR (equation 4 of the paper)
        LWLR = t1 * (t4 + t5) + t1* ((t9* R + t10* R + t11* R)/(1 - R*t3))

        # Calculate rho_app (Equation 5 in the paper)
        rho_app = LBOA / LWLR

        #Calculate LTOA (Equation 7 in the paper)
        LTOA = t1 * t2 + (t1 *(t8*rho_app + (t9 * rho_app + t10 * rho_app + t11 * rho_app)/(1-t3*R) )+ (t6 * F + t7 * F))/(1-t3 * R);

        # --- 5. Save Results ---, using pandas to write the data.
        df = pd.DataFrame({'Wavelength': wvlLUT.flatten(), 'LTOA': LTOA.flatten()}) #Ensure LTOA is correctly shaped.
        output_file = OUTPUT_DIR / f"LTOA_{case_name}.csv"
        df.to_csv(output_file, index=False)
        print(f"Saved LTOA for {case_name} to {output_file}")

    eng.quit()
    print("Processing complete.")

MATLAB paths added successfully
Number of simulations: 135


Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error usin

MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 0 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 1 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable:

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error usin

MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 6 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 7 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable:

...

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe



MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 114 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 115 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executa

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error usin

MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 123 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe


Skipping simulation 124 due to MODTRAN error.
MATLAB/MODTRAN error: 
  File /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/run_modtran_corrected.m, line 61, in run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executa

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

Error using run_modtran_corrected
MODTRAN execution failed: zsh:1: bad CPU type in executable: /Users/mirkomorello/Documents/Università/MSc_Sensors_Imaging/Final_Project/MODTRAN5/bin/Mod5_mac.exe

